# CMIP6Plus MIP table remapping
To support the extension of CMIP6Plus variable set the CMIP6 variables have been reallocated to a set of tables named by their modelling domain, frequency and variable shape,
e.g. atmos physical variables have moved from MIP table `Amon` to `APmon`, while atmospheric chemistry has moved to `ACmon` and aerosols to `AEmon`. Where data is on model levels `Lev` is appended to the table name.

The code below constructs a dictionary to map the name of the CMIP6 variable to the CMIP6Plus equivalent

In [2]:
import json
import glob

Load table files and ignore coordinate and generic CV information

In [3]:
table_files = glob.glob('../Tables/*')
# json_files_to_ignore =  [
#     'Tables/coordinate.json',
#     'Tables/formula_terms.json',
#     'Tables/generic_CV.json',
#     'Tables/grids.json']
# for i in json_files_to_ignore:
#     table_files.remove(i)

Loop over variables and extract the CMIP6 variable name from the mip table entry and add to a dictionary

In [4]:
cmip6_to_cmip6plus_variable_mapping = {}
for table_filename in table_files:
    table = json.load(open(table_filename))
    table_id = table['Header']['table_id']
    for varname, details in table['variable_entry'].items():
        if 'CMIP6' in details['provenance']:
            cmip6_table_id = details['provenance']['CMIP6']['mip_table']
            cmip6_varname = details['provenance']['CMIP6']['variable_name']
            cmip6_to_cmip6plus_variable_mapping[(cmip6_table_id, cmip6_varname)] = (table_id, varname)

Print the result

In [5]:
print(' #{(CMIP6 MIP table, CMIP6 Variable name): (CMIP6Plus MIP table, CMIP6Plus Variable name)}')
cmip6_to_cmip6plus_variable_mapping

 #{(CMIP6 MIP table, CMIP6 Variable name): (CMIP6Plus MIP table, CMIP6Plus Variable name)}


{('Ofx', 'areacello'): ('OPfx', 'areacello'),
 ('Ofx', 'basin'): ('OPfx', 'basin'),
 ('Ofx', 'deptho'): ('OPfx', 'deptho'),
 ('Ofx', 'hfgeou'): ('OPfx', 'hfgeou'),
 ('Ofx', 'masscello'): ('OPfx', 'masscello'),
 ('Ofx', 'sftof'): ('OPfx', 'sftof'),
 ('Ofx', 'thkcello'): ('OPfx', 'thkcello'),
 ('Ofx', 'ugrido'): ('OPfx', 'ugrido'),
 ('Ofx', 'volcello'): ('OPfx', 'volcello'),
 ('E1hr', 'psl'): ('AP1hrPt', 'psl'),
 ('E1hr', 'rlut'): ('AP1hrPt', 'rlut'),
 ('E1hr', 'ta'): ('AP1hrPt', 'ta'),
 ('E1hr', 'ta27'): ('AP1hrPt', 'ta27'),
 ('E1hr', 'tntr27'): ('AP1hrPt', 'tntr27'),
 ('E1hr', 'ua'): ('AP1hrPt', 'ua'),
 ('E1hr', 'ua27'): ('AP1hrPt', 'ua27'),
 ('E1hr', 'utendnogw27'): ('AP1hrPt', 'utendnogw27'),
 ('E1hr', 'va'): ('AP1hrPt', 'va'),
 ('E1hr', 'va27'): ('AP1hrPt', 'va27'),
 ('E1hr', 'vtendnogw27'): ('AP1hrPt', 'vtendnogw27'),
 ('E1hr', 'wap'): ('AP1hrPt', 'wap'),
 ('E1hr', 'wap27'): ('AP1hrPt', 'wap27'),
 ('E1hr', 'zg27'): ('AP1hrPt', 'zg27'),
 ('CFday', 'cl'): ('APdayLev', 'cl'),
 ('CFday

Table mapping

In [12]:

from collections import defaultdict
table_mapping = defaultdict(list)
for table_filename in table_files:
    table = json.load(open(table_filename))
    table_id = table['Header']['table_id']
    # table_mapping[table_id] = defaultdict(list)
    for varname, details in table['variable_entry'].items():
        if 'CMIP6' in details['provenance']:
            cmip6_table_id = details['provenance']['CMIP6']['mip_table']
            if table_id not in table_mapping[cmip6_table_id]:
                table_mapping[cmip6_table_id].append(table_id)

# cmip6+ : cmip6
print(table_mapping)

defaultdict(<class 'list'>, {'Ofx': ['OPfx'], 'E1hr': ['AP1hrPt', 'AP1hr'], 'CFday': ['APdayLev', 'APday'], 'Esubhr': ['APsubhrPt', 'AEsubhrPt', 'APsubhrPtLev', 'APsubhrPtSite', 'AEsubhrPtSite'], 'IfxGre': ['GIGfx'], 'Oclim': ['OPmonClimLev', 'OPmonClim'], 'Eday': ['LPday', 'APday', 'OPday', 'LIday'], 'day': ['LPday', 'APday', 'LIday'], 'SImon': ['SImon', 'SImonPt'], 'E3hr': ['LP3hr', 'AP3hr'], '3hr': ['LP3hr', 'OP3hrPt', 'AP3hr', 'LP3hrPt', 'AP3hrPt'], 'Amon': ['APmonLev', 'LImon', 'APmonClimLev', 'APmon', 'APmonClim'], 'CFmon': ['APmonLev', 'APmon'], 'Emon': ['APmonLev', 'ACmon', 'OBmonLev', 'OPmon', 'APmon', 'AEmon', 'LPmon', 'AEmonLev', 'OBmon', 'OPmonLev'], 'AERmon': ['APmonLev', 'AEmon', 'AEmonLev'], 'Oyr': ['OByr', 'OPyr', 'OPyrLev', 'OByrLev'], 'IyrAnt': ['GIAyr'], '6hrPlevPt': ['LI6hrPt', 'AP6hrPt', 'AE6hrPt', 'LP6hrPt'], 'Eyr': ['LPyr', 'OPyrLev', 'LPyrPt'], 'LImon': ['LImon', 'LPmon'], 'Oday': ['OBday', 'OPday'], 'EdayZ': ['APdayZ'], 'Omon': ['OBmonLev', 'OPmonZ', 'OPmon', '

In [13]:
json.dump(table_mapping,open('table_6_to_6plus.json','w'),indent=2)